In [1]:
from datasets import load_dataset
from transformers import (
    MarianTokenizer,
    MarianMTModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import evaluate

In [2]:
# Load dataset and subset
print("Loading WMT14 dataset...")
dataset = load_dataset("wmt14", "de-en")
raw_dataset = dataset["train"].select(range(2500))
split = raw_dataset.train_test_split(test_size=0.1)
train_raw = split["train"]
val_raw = split["test"]

# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

Loading WMT14 dataset...


/opt/anaconda3/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:

# Preprocess with batching
def preprocess_batch(batch):
    translations = batch["translation"]
    en_texts = [item["en"] for item in translations]
    de_texts = [item["de"] for item in translations]
    model_inputs = tokenizer(en_texts, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(de_texts, truncation=True, padding="max_length", max_length=128)["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

print("Preprocessing data...")
train_dataset = train_raw.map(preprocess_batch, batched=True)
val_dataset = val_raw.map(preprocess_batch, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# BLEU scoring
bleu = evaluate.load("bleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]  # BLEU expects list of lists
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    preds, labels = postprocess_text(decoded_preds, decoded_labels)
    result = bleu.compute(predictions=preds, references=labels)
    return {"bleu": result["bleu"]}

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian-finetuned-en-de",
    per_device_train_batch_size=4,
    num_train_epochs=2,
    save_strategy="epoch",
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Preprocessing data...


Map:   0%|          | 0/2250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/qx/kwt2yzc57djflb63h7j_sxg40000gn/T/ipykernel_73921/2298876441.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Train and save
print("Training model...")
trainer.train()
model.save_pretrained("./marian-finetuned-en-de")
tokenizer.save_pretrained("./marian-finetuned-en-de")
print("\n Training complete and model saved to ./marian-finetuned-en-de")

In [6]:

# Save your fine-tuned model
model.save_pretrained("./marian-finetuned-en-de")
tokenizer.save_pretrained("./marian-finetuned-en-de")

('./marian-finetuned-en-de/tokenizer_config.json',
 './marian-finetuned-en-de/special_tokens_map.json',
 './marian-finetuned-en-de/vocab.json',
 './marian-finetuned-en-de/source.spm',
 './marian-finetuned-en-de/target.spm',
 './marian-finetuned-en-de/added_tokens.json')